Load Julia packages (libraries)

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');
df = convert(DataFrame, d);

dcc = filter(row -> !(ismissing(row[:rgdppc_2000])), df)
dcc[:log_gdp] = log.(dcc[:rgdppc_2000])
dcc[:cont_africa] = Array{Float64}(convert(Array{Int}, dcc[:cont_africa]))
dcc[:rugged] = convert(Array{Float64}, dcc[:rugged])
first(dcc[[:rugged, :cont_africa, :log_gdp]], 5)

,rugged,cont_africa,log_gdp
,Float64,Float64,Float64
1,0.858,1.0,7.49261
2,3.427,0.0,8.21693
3,0.769,0.0,9.93326
4,0.775,0.0,9.40703
5,2.688,0.0,7.79234


Define the Stan language model

In [4]:
m_8_1 = "
data{
    int N;
    vector[N] log_gdp;
    vector[N] cont_africa;
    vector[N] rugged;
    vector[N] rugged_cont_africa;
}
parameters{
    real a;
    real bR;
    real bA;
    real bAR;
    real sigma;
}
model{
    vector[N] mu = a + bR * rugged + bA * cont_africa + bAR * rugged_cont_africa;
    sigma ~ uniform( 0 , 10 );
    bAR ~ normal( 0 , 10 );
    bA ~ normal( 0 , 10 );
    bR ~ normal( 0 , 10 );
    a ~ normal( 0 , 100 );
    log_gdp ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_8_1",
monitors = ["a", "bR", "bA", "bAR", "sigma"],
model=m_8_1, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/scripts/08/tmp/m_8_1.stan will be updated.



Input data for cmdstan

In [6]:
m_8_1_data = Dict("N" => size(dcc, 1),
"log_gdp" => dcc[:log_gdp],  "rugged" => dcc[:rugged],
"cont_africa" => dcc[:cont_africa],
"rugged_cont_africa" => dcc[:rugged] .* dcc[:cont_africa] );

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_8_1_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.22 0.14  9.00  9.46   282    1
bR    -0.21 0.08 -0.33 -0.08   275    1
bA    -1.94 0.24 -2.33 -1.59   268    1
bAR    0.40 0.14  0.18  0.62   271    1
sigma  0.96 0.05  0.87  1.04   339    1
"


Inference for Stan model: m_8_1_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.36, 0.37, 0.38, 0.37) seconds, 1.5 seconds total
Sampling took (0.41, 0.37, 0.41, 0.34) seconds, 1.5 seconds total

                 Mean     MCSE  StdDev     5%    50%       95%    N_Eff  N_Eff/s    R_hat
lp__              -76  4.4e-02     1.7    -79    -76  -7.4e+01  1.4e+03  9.4e+02  1.0e+00
accept_stat__    0.92  2.4e-03   0.099   0.72   0.96   1.0e+00  1.6e+03  1.1e+03  1.0e+00
stepsize__       0.34  3.1e-02   0.043   0.30   0.34   4.1e-01  2.0e+00  1.3e+00  7.0e+13
treedepth__       3.1  5.4e-02    0.63    2.0    3.0   4.0e+00  1.3e+02  8.8e+01  1.0e+00
n_leapfrog__       11  5.5e-01     5.8    3.0    7.0   2.3e+01  1.1e+02  7.2e+01  1.0e+00
divergent__      0.00      nan    0.00   0.00   0.00   0.0e+00      nan      nan      nan
energy__           78  6.3e-02     2.3     75     78   8.3e+01  1.3e+03  8.7e+02  1.0e+00
a   

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.22 0.14  9.00  9.46   282    1\nbR    -0.21 0.08 -0.33 -0.08   275    1\nbA    -1.94 0.24 -2.33 -1.59   268    1\nbAR    0.40 0.14  0.18  0.62   271    1\nsigma  0.96 0.05  0.87  1.04   339    1\n"

Describe the draws

In [8]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE        MCSE      ESS
    a  9.22257912 0.141746516 0.0022412092 0.00348867217 1000
   bR -0.20164179 0.080405223 0.0012713182 0.00190226343 1000
   bA -1.94557467 0.232540525 0.0036767885 0.00595172288 1000
  bAR  0.39089751 0.134362777 0.0021244620 0.00321724135 1000
sigma  0.95385603 0.054803606 0.0008665211 0.00090770227 1000

Quantiles:
          2.5%       25.0%       50.0%      75.0%       97.5%   
    a  8.93248375  9.12892750  9.2216100  9.31639750  9.49624025
   bR -0.36018892 -0.25604350 -0.2024435 -0.14636325 -0.04405355
   bA -2.40636250 -2.10265750 -1.9479150 -1.79021500 -1.48831625
  bAR  0.12463238  0.30056875  0.3908600  0.47934275  0.65347090
sigma  0.85032590  0.91669825  0.9507275  0.98894725  1.07046500



End of `m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*